# Practicum: Modern Hopfield Network Spell Checking and Word Recommendation
___
In this practicum problem, we'll implement a modern Hopfield Network and use it for spell checking and word recommendation tasks.

* _What are Hopfield Networks_? Hopfield Networks are used for associative memory, where the network can recall a pattern from a partial input. The modern version of Hopfield Networks uses continuous values instead of binary values, and it can store multiple patterns. 
* We'll use the following paper to guide our implementation and analysis: [Ramsauer, H., Schafl, B., Lehner, J., Seidl, P., Widrich, M., Gruber, L., Holzleitner, M., Pavlovi'c, M., Sandve, G.K., Greiff, V., Kreil, D.P., Kopp, M., Klambauer, G., Brandstetter, J., & Hochreiter, S. (2020). Hopfield Networks is All You Need. ArXiv, abs/2008.02217.](https://arxiv.org/abs/2008.02217)

## Tasks
Before we get started, we'll quickly review modern Hopfied Networks. Then, you'll execute the `Run All Cells` command to check if you (or your neighbor) have any code or setup issues. Code issues, then raise your hands - and let's get those fixed!

* __Task 1: Setup, Data, Constants (5 min)__: Let's take 5 minutes to load [a Simpsons character library from Kaggle](https://www.kaggle.com/datasets/kostastokis/simpsons-faces) that our Hopfield network will memorize.
*  __Task 2: Build a Modern Network Model (5 min)__: In this task, we'll formulate the image dataset we give the network and then create a model of a modern Hopfield network. We'll also quickly check to ensure we are doing what we think we are doing.
* __Task 3: Retrieve a memory from the network (30 min)__: In this task, we will retrieve a memory from the modern Hopfield network starting from a random state vector $\mathbf{s}_{\circ}$. We'll corrupt an image (by cutting off some fraction of the image) and then see if the model recovers the correct memory given the corrupted starting point. 

Let's get started!

___

## Background
A modern Hopfield network addresses many of the perceived limitations of the original Hopfield network. The original Hopfield network was limited to binary values and could only store a limited number of patterns. The modern Hopfield network uses continuous values and can store a large number of patterns.
* For a detailed discussion of the key milestones in the development of modern Hopfield networks, check out [Hopfield Networks is All You Need Blog, GitHub.io](https://ml-jku.github.io/hopfield-layers/)

### Algorithm
The user provides a set of memory vectors $\mathbf{X} = \left\{\mathbf{x}_{1}, \mathbf{x}_{2}, \ldots, \mathbf{x}_{m}\right\}$, where $\mathbf{x}_{i} \in \mathbb{R}^{n}$ is a memory vector of size $n$ and $m$ is the number of memory vectors. Further, the user provides an initial _partial memory_ $\mathbf{s}_{\circ} \in \mathbb{R}^{n}$, which is a vector of size $n$ that is a partial version of one of the memory vectors and specifies the _temperature_ $\beta$ of the system.

__Initialize__ the network with the memory vectors $\mathbf{X}$, and the inverse temperature $\beta$. Set current state to the initial state $\mathbf{s} \gets \mathbf{s}_{\circ}$

Until convergence __do__:
   1. Compute the _current_ probability vector defined as $\mathbf{p} = \texttt{softmax}(\beta\cdot\mathbf{X}^{\top}\mathbf{s})$ where $\mathbf{s}$ is the _current_ state vector, and $\mathbf{X}^{\top}$ is the transpose of the memory matrix $\mathbf{X}$.
   2. Compute the _next_ state vector $\mathbf{s}^{\prime} = \mathbf{X}\mathbf{p}$ and the _next_ probability vector $\mathbf{p}^{\prime} = \texttt{softmax}(\beta\cdot\mathbf{X}^{\top}\mathbf{s}^{\prime})$.
   3. If $\mathbf{p}^{\prime}$ is _close_ to $\mathbf{p}$ or we run out of iterations, then __stop__. For example, $\lVert \mathbf{p}^{\prime} - \mathbf{p}\rVert_{2}^{2} \leq \epsilon$ for some small $\epsilon > 0$.
   4. Otherwise, update the state $\mathbf{s} \gets\mathbf{s}^{\prime}$, and __go back to__ step 1.

   
This algorithm is implemented in [the `recover(...)` method](src/Compute.jl).

## Task 1: Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl"); # load a bunch of libs, including the ones we need to work with images

  Activating project at `~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Practicum-S2025`
    Updating `~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Practicum-S2025/Project.toml`
  [336ed68f] + CSV v0.10.15
  [aaaa29a8] + Clustering v0.15.8
  [5ae59095] + Colors v0.13.0
  [a93c6f00] + DataFrames v1.7.0
  [5789e2e9] + FileIO v1.17.0
  [033835bb] + JLD2 v0.5.13
  [872c559c] + NNlib v0.9.30
  [91a5bcdd] + Plots v1.40.13
  [08abe8d2] + PrettyTables v2.4.0
  [10745b16] + Statistics v1.11.1
  [24678dba] + TSne v1.3.0
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
  [9a3f8284] ~ Random ⇒ v1.11.0
  [8dfed614] ~ Test ⇒ v1.11.0
    Updating `~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Practicum-S2025/Manifest.toml`
  [79e6a3ab] + Adapt v4.3.0
  [66dad0bd] + AliasTables v1.1.3
  [a9b6321e] + Atomix v1.1.1
  [d1d4a3ce] + BitFlags v0.1.9
  [336ed68f] + CSV v0.10.15
  [d360d2e6] + ChainRulesCore v1.25.1
  [aaaa29a8] + Clustering v0.15.8
  [944b1d66] + CodecZlib v0.7.8
  [35d6a980] + ColorSchemes v3.2

### Constants
Before we load the data, let's set up some constants that we will use in the exercise.

In [2]:
number_of_words_to_memorize = 24; # how many images do we want to memorize?
number_of_embedding_dimesions = 50; # number of embedding dimensions for each word
β = 0.5; # Inverse temperature of the system (high T -> low β)

### Data
We'll use the [GloVe pretrained word embedding dataset](https://nlp.stanford.edu/projects/glove/) to as the memories for our Hopfield model. The **GloVe (Global Vectors for Word Representation)** dataset is a widely used pre-trained word embedding resource developed by Pennington, Socher, and Manning at Stanford University. 
* _What is it?_ It constructs vector representations of words by aggregating global word co-occurrence statistics from a corpus, enabling semantic relationships to be captured in vector space. GloVe embeddings have been trained on large datasets such as Wikipedia, Gigaword, and Common Crawl, offering dimensionalities typically ranging from 50 to 300. These embeddings are foundational for many NLP tasks, including text classification, sentiment analysis, and machine translation.
* See: [Pennington et al., EMNLP 2014, GloVe: Global Vectors for Word Representation](https://aclanthology.org/D14-1162/) for more details on this dataset.

This dataset is large, so we won't check it into the repository. Instead, we'll download it from the internet. Fill me in.

In [3]:
word2vec, vec2word = let

    # do we have the embeddings downloaded?
    data = nothing;
    if (isfile(joinpath(_PATH_TO_DATA, "glove_6B_50d.jld2")) == false) 
        # TODO: download logic goes here ...
    else
        data = JLD2.load(joinpath(_PATH_TO_DATA, "glove_6B_50d.jld2")) # Ok, we have the embeddings file, so let's load it
    end

    # load the embeddings
    word2vec = data["word2vec"] # this is a Dict{String, Tuple{Float32}}
    vec2word = data["vec2word"] # this is a Dict{Tuple{Float32}, String}

    # return -
    (word2vec, vec2word)
end;

In [4]:
word2vec

Dict{String, NTuple{50, Float64}} with 400000 entries:
  "newdigate"   => (-0.13302, 0.99457, 0.19927, 0.062554, 0.40293, 0.58369, 0.1…
  "daufuskie"   => (0.47702, -0.33101, -1.0066, 0.70388, -0.29973, -0.54381, -0…
  "single-arm"  => (0.7153, 0.67314, 1.6732, 0.21066, 1.1875, 2.3162, 0.078471,…
  "titration"   => (0.30649, -0.66054, -0.27341, -0.30447, -0.58893, 0.64415, 1…
  "qajar"       => (-1.2141, -0.039392, -0.63721, 0.058088, -0.22625, -1.0897, …
  "pinheiro"    => (0.49711, -0.31449, -1.0776, 0.61769, 0.066749, -2.1997, 0.0…
  "hospitalet"  => (-0.55697, 0.19924, -1.099, -0.92013, -1.1645, -0.45907, 0.7…
  "kennedale"   => (-0.91243, -0.097526, 0.031527, 0.71883, -0.89733, -1.0061, …
  "tetracyclic" => (0.49991, -0.98145, 0.01992, -0.88848, 0.24377, 0.27405, 0.8…
  "moher"       => (0.097974, 0.99096, -0.63521, 0.49702, 0.043669, 0.035202, 0…
  "entomb"      => (0.64585, -0.28799, 0.59975, -0.43203, -0.64485, 0.11591, -0…
  "vanderwerff" => (-1.0542, -0.47031, -1.4568, 0.9361

Fill me in.

In [5]:
test_words, test_vocabulary = let

    # initialize -
    vocabulary = Array{Float64,2}(undef, number_of_words_to_memorize, number_of_embedding_dimesions); # this is a matrix of Float64
    test_words = Array{String,1}(undef, number_of_words_to_memorize); # this is a vector of strings
    total_number_of_words = length(word2vec); # this is the total number of words in the dataset
    index_of_words_to_learn = randperm(total_number_of_words)[1:number_of_words_to_memorize]; # this is the random index of words to learn

    # get the keys of word2vec -
    words = keys(word2vec) |> collect; # this is a vector of strings

    # loop over the words to learn
    for i ∈ eachindex(index_of_words_to_learn)
        
        j = index_of_words_to_learn[i]; # this is the index of the word we want to learn
        wⱼ = words[j]; # this is the word we want to learn
        test_words[i] = wⱼ; # this is the word we want to learn 
        embedding = word2vec[wⱼ]; # this is the embedding of the word we want to learn

        for j ∈ 1:number_of_embedding_dimesions
            vocabulary[i,j] = embedding[j]; # this is the embedding of the word we want to learn
        end
    end

    test_vocabulary = vocabulary |> transpose |> Matrix; # this is the vocabulary we want to learn

    # return -
    (test_words, test_vocabulary);
end;

In [6]:
test_words

24-element Vector{String}:
 "klingenthal"
 "riedelsheimer"
 "newpaper"
 "vixen"
 "botticelli"
 "mtbe"
 "bodian"
 "thiocyanate"
 "riles"
 "strad"
 ⋮
 "klitschkos"
 "dustup"
 "activités"
 "rybarikova"
 "2125"
 "sippola"
 "3634"
 "arimori"
 "58-58"

__Let's do some fun stuff with embeddings.__ Fill me in.

## Task 2: Can we recover an uncorrupted memory?
In this task, we'll formulate the set of word embeddings (memories) that we feed to a modern Hopfield network. First, we'll create a modern Hopfield network model, and then we'll use the model to retrieve a memory from the network. We'll do this by starting from a random state vector $\mathbf{s}_{\circ}$, which is a partial version of one of the memory vectors, i.e., a misspled word. We'll corrupt the word by randomly permuating it's embeddings, and then we'll see if the model recovers the correct word (memory) given the corrupted starting point.

### Model
Let's start by creating a model of a modern Hopfield network. 
* We'll construct [a `MyModernHopfieldNetworkModel` instance](src/Types.jl) using a custom [`build(...)` function](src/Factory.jl). The [`build(...)` method](src/Factory.jl) takes the type of thing we want to build, the (linearized) image library we want to encode, and the (inverse) system temperature $\beta$ as inputs — images along the columns.
* The [`build(...)` function](src/Factory.jl) returns a `MyModernHopfieldNetworkModel` instance, where the image library is stored in the `X::Array{Float64,2}` field, and the system temperature is stored in the `β::Float64` field.

We'll store the Hopfield network instance in the `mymodel::MyModernHopfieldNetworkModel` variable.

In [7]:
mymodel = let

    # initialize -
    memorycollection =test_vocabulary; # words (memories) on columns
    index_vector = 1:number_of_words_to_memorize |> collect; # this is the index of the words we want to learn
    words = keys(test_vocabulary) |> collect; # this is a vector of strings
    
    # build model -
    model = build(MyModernHopfieldNetworkModel, (
            memories = memorycollection, # this is the data we want to memorize. Images on columns
            β = β, # Inverse temperature of the system. A big beta means we are more likely to get the right answer
    ));

    model; # return the model to the calling scope
end;

__Check__: Let's do a quick check to make sure we are doing what we think we are doing. Let's check what's stored in the columns of the `model.X` field. These should be the words that we are encoding into the Hopfield network.

In [8]:
let
    
    X = mymodel.X; # get the training data in the model
    index_to_check = rand(1:number_of_words_to_memorize); # what index do we want to check?
    
    
    eᵢ = X[:,index_to_check] |> Tuple # this is the embedding of the word we want to learn
    wᵢ = test_words[index_to_check]; # this is the word we want to learn
    ŵᵢ = vec2word[eᵢ]; # this is the word we think we learned

    println("The word at index $(index_to_check) that we encoded is: $(wᵢ). The word from vec2word is: ", ŵᵢ);
end

The word at index 18 that we encoded is: activités. The word from vec2word is: activités


### Retrieve a memory from the network
Next, we'll test if we can recover uncorrupted and corrupted memories from the Hopfield network.

* _What should we expect_: We are _guaranteed_ that the network will converge to a local minimum, but we are not guaranteed that the local minimum is the same as the original image. 

Let's start by specifying which memory we are trying to recover in the `memoryindextorecover::Int` variable.

In [9]:
memoryindextorecover = 15; # which memory vector will we choose?

Next, let's build an uncorrupted and corrupted initial condition vector using the true word emebedding vector. We'll store 
the uncorrupted word in the `sₒ::Array{Float64,1}` variable, while the corrupted word will be stored in the `s₁::Array{Float64,1}` variable. Let's start with the uncorrupted memory.

In [10]:
sₒ = mymodel.X[:,memoryindextorecover]; # this is the memory vector we want to recover

What word does `memoryindextorecover::Int64` point to?

In [11]:
let 
    X = mymodel.X; # get the training data in the model
    p = β*(transpose(X) * sₒ) |> s-> NNlib.softmax(s) # this is the probability of the word we want to learn
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned
    w = test_words[memoryindextorecover]; # this is the word we want to learn
    println("The word at index $(memoryindextorecover) that we (think) we encoded is: $(ŵ). Check: the true word is: ", w);
end

The word at index 15 that we (think) we encoded is: chikka. Check: the true word is: chikka


Now that we have a starting memory encoded in the state vector $\mathbf{s}_{\circ}$, can we recover the original uncorrupted word? We are guaranteed an image, but maybe _not_ the correct one.
* _Implementation_: We implemented the modern Hopfield recovery algorithm above in [the `recover(...)` method](src/Compute.jl). This method takes our `model::MyModernHopfieldNetworkModel` instance, the initial configuration vector `sₒ::Array{Int32,1}`, and the maximum number `maxiterations::Int64`, and iteration tolerance parameter `ϵ::Float64`. 
* [The `recover(...)` method](src/Compute.jl) returns the recovered image in the e`s₁::Array{Float32,1}` variable, the image at each iteration in the `f::Dict{Int, Array{Float32,2}}` dictionary, and the probability of the image at each iteration in the `p::Dict{Int, Array{Float32,2}}` variable. The frames and probability dictionaries are indexed from `0`.

In [12]:
(ŝₒ,fₒ,pₒ) = recover(mymodel, sₒ, maxiterations = 10000, ϵ = 1e-16); # iterate until we hit stop condition

How many iterations did it take to converge? 

In [13]:
println("How many iterations: $(length(fₒ))") # how many iterations did we need to converge?

How many iterations: 13


Fill me in.

In [14]:
recovered_word_uncorrupted = let 
    
    # initialize -
    number_of_iterations = length(fₒ); # how many iterations did we need to converge? 
    p = pₒ[number_of_iterations - 1]; # this is the probability of the word we want to learn  (0 based)
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned
    
    ŵ; # return the word we *think* we learned
end

"chikka"

__Check__: Let's check to see if the recovered image is identical to the original image (not guaranteed). We can do this by checking the `s₁::Array{Float32,1}` variable against the original image.

In [15]:
let
    true_word = test_words[memoryindextorecover]; # this is the word we want to learn
    @assert recovered_word_uncorrupted == true_word
end

## Task 3: Retrieve a corrupted memory from the network
Fill me in.

Let's build the corrupted memory. We'll iterate through each embedding dimension from the uncorrupted word; sometimes, we'll make a mistake and replace the correct embedding value with an incorrect value.
* _What is the $\theta$ parameter_? The $\theta$ hyperparameter controls how often we make mistakes. Its interpretation depends upon our _mistake_ model. For example, if we are cutting off some fraction of the embedding dimension, then $\theta$ describes the fraction of the image we are cutting off. Alternatively, if we add noise, $1 - \theta$ describes the fraction of the original image we are keeping.

Whichever model we use, the $\theta\in[0,1]$. We store the corrupted image in the `s₁::Array{Float64,1}` variable.

In [16]:
s₁ = let

    # initialize -
    sₒ = mymodel.X[:,memoryindextorecover]; # this is the memory vector we want to recover (uncorrupted)
    s₁ = Array{Float32,1}(undef, number_of_embedding_dimesions); # initialize some space to store the corrupted word
    θ = 0.65; # threshold (fraction 1 - θ is the fraction the original memory that we retain in model 2)

    # Corruption model: Cutoff part of the memory
    cutoff = (1-θ)*number_of_embedding_dimesions |> x-> round(Int,x);
    for i ∈ 1:number_of_embedding_dimesions
        eᵢ =  sₒ[i]; # We have some gray-scale values in the original vector, need to perturb
        if (i ≤ cutoff)
            s₁[i] = eᵢ;
        else
            s₁[i] = β*randn(); # add some random noise (proportional to β)
        end
    end
    
    s₁ # return corrupted data to the calling scope
end;

How different is the corrupted memory from the original (true) memory? Under a squared L2 measure, the distance between the two memories is given by $\lVert \mathbf{s}_{\circ} - \mathbf{s}_{1}\rVert_{2}^{2}$. What is this distance?

In [17]:
let
    Δ = norm(s₁ - sₒ)^2; # this is the L2 squared difference between the corrupted and uncorrupted memory
    println("The squared L2 norm difference between the corrupted and uncorrupted image is: $(Δ)");
end

The squared L2 norm difference between the corrupted and uncorrupted image is: 24.8718443278219


Under the squared L2 measure, what is the closest memory to the corrupted memory?

In [18]:
let
    # initialize -
    X = mymodel.X; # get the training data in the model
    Δ = Array{Float64,1}(undef, number_of_words_to_memorize); 

    for i ∈ 1:number_of_words_to_memorize
        sᵢ = X[:,i]; # this is the memory vector we want to recover
        Δ[i] = norm(s₁ - sᵢ)^2; # this is the L2 squared difference between the corrupted and uncorrupted memory
    end

    # compute the most probable word using the L2 measure -
    p = NNlib.softmax(Δ); # this is the probability of the word we want to learn
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned

    println("The closest (using a sq-L2 measure) word to the corrupted memory is: $(ŵ)");
end

The closest (using a sq-L2 measure) word to the corrupted memory is: rybarikova


What about using the inner product (self attention) measure? The distance between the two memories is given by $\beta\cdot\langle \mathbf{s}_{\circ}, \mathbf{s}_{1}\rangle$. What is the most probable memory to the corrupted memory?

In [19]:
let
    # initialize -
    X = mymodel.X; # get the training data in the model
    p = β*(transpose(X) * s₁) |> s-> NNlib.softmax(s) # this is the probability of the word we want to learn
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned

    # make a table -

    println("The closest word (using self-attention socre) to the corrupted memory is: $(ŵ)");
end

The closest word (using self-attention socre) to the corrupted memory is: chikka


__Do we converge to the correct word starting from corrupted word__? Now that we have a starting (corrupted) memory encoded in the $\mathbf{s}_{1}$ state vector, can we recover the original uncorrupted memory, i.e., the uncorrputed word? We are guaranteed to converge to a word, but maybe _not_ the correct one.
* _Implementation_: We implemented the modern Hopfield recovery algorithm above in [the `recover(...)` method](src/Compute.jl). This method takes our `model::MyModernHopfieldNetworkModel` instance, the initial configuration vector `sₒ::Array{Int32,1}`, and the maximum number `maxiterations::Int64`, and iteration tolerance parameter `ϵ::Float64`. 
* [The `recover(...)` method](src/Compute.jl) returns the recovered image in the e`s₁::Array{Float32,1}` variable, the image at each iteration in the `f::Dict{Int, Array{Float32,2}}` dictionary, and the probability of the image at each iteration in the `p::Dict{Int, Array{Float32,2}}` variable. The frames and probability dictionaries are indexed from `0`.

In [20]:
(ŝ₁,f₁,p₁) = recover(mymodel, s₁, maxiterations = 10000, ϵ = 1e-16); # iterate until we hit stop condition

In [21]:
p₁[1][15]

0.32640870499738117

How many iterations did it take to converge? 

In [22]:
println("The network converged to an answer (staring from a corrupted word) in: $(length(f₁)) iterations.") # how many iterations did we need to converge?

The network converged to an answer (staring from a corrupted word) in: 9 iterations.


In [23]:
recovered_word_corrupted = let 
    
    # initialize -
    number_of_iterations = length(f₁); # how many iterations did we need to converge? 
    p = p₁[number_of_iterations - 1]; # this is the probability of the word we want to learn  (0 based)
    ŵ = argmax(p) |> i-> test_words[i]; # this is the index of the word we think we learned
    
    ŵ; # return the word we *think* we recovered
end

"kaoliang"